### Before you begin

To check whether the code you've written is correct, we'll again use automark. Please update the username in the cell below with your student number. 


### Troubleshooting

In the Spark tasks in this assignment, Spark might randomly crash if it doesn't have enough memory available. If you encounter issues like that when working on the tasks today, we suggest increasing the amount of memory available to the docker image to 4gb or more. If you are using docker-for-windows or docker-for-mac, you can easily increase it from the Whale 🐳 icon in the task bar, then go to Preferences -> Resources.

Note that passing a command line argument like `-m 4g` when running `docker run ...` does only work for limiting the memory to even lower values than the limit in your Docker Resource Preferences, and does not work for relaxing the limit that is set in these Preferences. For more information, please refer to the Docker [documentation](https://docs.docker.com/config/containers/resource_constraints/).

In [8]:
import automark as am
import math

# fill in you student number as your username
am.configure(username='14090759')

# to check your progress, you can run this function
am.get_progress()

----------------------------------------------------------------------
| Murong, Yanchao                                                    |
----------------------------------------------------------------------
| a0_t1_sailor_avg_experience                       | completed      |
| a0_t2_sailor_avg_experience_pyspark               | completed      |
| a1a_t1_all_model_number_b                         | completed      |
| a1a_t2_all_model_number_not_b                     | completed      |
| a1a_t3_avg_PCs_speed                              | completed      |
| a1a_t4_max_printer_price                          | completed      |
| a1a_t5_min_spending_visa                          | completed      |
| a1a_t6_customer_id                                | completed      |
| a1a_t7_highest_price_USD                          | completed      |
| a1b_t1a_at_least_four_products                    | completed      |
| a1b_t1b_decending_speed_laptop                    | completed      |
| a1b_

# Part A - MapReduce

## Our MapReduce Engine

Real-world MapReduce systems like [Apache Hadoop](https://hadoop.apache.org/) are tedious to install and debug, and require code in a programming language like Java. For the sake of simplicity and to allow for easy debugging, we work with our own local, non-parallel MapReduce engine written in Python. Note that your MapReduce programs (if written correctly) would also work in distributed mode on a real cluster.

In the following, we ask you to implement several mapreduce programs and run them via this simple MapReduce engine.

In [ ]:
def mapreduce(input_partitions, f_map, f_reduce, num_reducers=2, print_debug_text=False):
    if print_debug_text: print("---Starting MAP-phase.---")
    # These arrays will hold the outputs of the map-phase for each input partition
    map_output_partitions = []
    # We are running the map-phase on each input partition now. In a real mapreduce system, this would run
    # in parallel on different machines.
    for counter, input_partition in enumerate(input_partitions):
        # suppose input_partition contains a list of documents
        if print_debug_text: print(f"  Applying f_map to input partition {counter}")
        map_output = []
        # We apply f_map to each (key, value) pair in the input partition, and store the corresponding outputs
        for key, value in input_partition:
            # suppose that key refers to doc_id and value refers to document content
            if print_debug_text: print(f"    f_map({key}, {value}) -> ")
            for mapped_key, mapped_value in f_map(key, value):
                # suppose mapped_key is the token and mapped_value is 1 (count to be reduced)
                if print_debug_text: print(f"      ({mapped_key}, {mapped_value})")
                map_output.append((mapped_key, mapped_value))
        # Store output partition of this map operation        
        map_output_partitions.append(map_output)        
    
    # Next, we start the shuffle phase. We need to create several reducer partitions from the map outputs, 
    # assign each key to a partition and collect all the values for this key.
    if print_debug_text: print("\n---Starting SHUFFLE-phase.---")
    reduce_input_partitions = []
    # We create as many reducer partitions as specified by num_reducers
    for _ in range(0, num_reducers):
        target_partition = dict()
        reduce_input_partitions.append(target_partition)
        
    # We shuffle each map output partition now. In a real mapreduce system, this would run
    # in parallel on different machines.
    for counter, map_output_partition in enumerate(map_output_partitions):
        if print_debug_text: print(f"  Shuffling map output input partition {counter}")    
        # We process each (key, value) pair from the map-output here
        for key, value in map_output_partition:
            # suppose key here is the token, value is the count(1)
            # We determine the target partition for this key via hash-partitioning
            target_partition_index = abs(hash(key)) % num_reducers
            if print_debug_text: print(f"    Assigning key [{key}] to reducer input {target_partition_index} via hash-partitioning")            
            # We add the value to the group of the key in the target partition
            target_partition = reduce_input_partitions[target_partition_index]
            if not key in target_partition:
                target_partition[key] = []            
            target_partition[key].append(value)
            # target_partition would contain {token: [1,1,1,1, ...],...}
            # reduce_input_partitions contains list of target_partition, one target_partition corresponds to one reducer
    
    # Next, we run the reduce-phase
    if print_debug_text: print("\n---Starting REDUCE-phase.---")
    reduce_output_partitions = []
        
    # We reduce each reduce partition now. In a real mapreduce system, this would run
    # in parallel on different machines.            
    for counter, reduce_input_partition in enumerate(reduce_input_partitions):
        # suppose reduce_input_partition refers to the list of postings(token, 1) for given partition
        reduce_output = [] # it will contain a list of tuples (token, count) for given partition
        if print_debug_text: print(f"  Applying f_reduce to reduce_input partition {counter}")    
        # We apply f_reduce to each key and its corresponding values now and collect the results
        for key, values in reduce_input_partition.items():
            # suppose key is the token, values is the count list [1,1,1,1,...]
            if print_debug_text: print(f"    f_reduce({key}, {values}) -> ")
            for reduced_key, reduced_value in f_reduce(key, values):
                if print_debug_text: print(f"      ({reduced_key}, {reduced_value})")
                reduce_output.append((reduced_key, reduced_value))
        reduce_output_partitions.append(reduce_output)
        
    return reduce_output_partitions    # mutilple list of tuples (token, count) for each partition

The following is a helper function for local testing, which checks the size of the result of a mapreduce job and tests where certain keys are present and have the expected value.

In [ ]:
def verify_locally(mapreduce_result, expected_num_keys, expected_keys_and_values):
    print("\nRESULT VERIFICATION:")
    results_by_key = {}
    for partition in mapreduce_result:
        for key, value in partition:
            results_by_key[key] = value

    if len(results_by_key) != expected_num_keys:
        print(f"  ERROR: Expected {len(expected_keys_and_values)} keys in the output, but found {len(results_by_key)}!")

    for key_of_interest, expected_value in expected_keys_and_values:
        if key_of_interest not in results_by_key:
            print(f"  ERROR: Unable to find key [{key_of_interest}] in result!")
        else:
            observed_value = results_by_key[key_of_interest]
            if isinstance(expected_value, (float, int)):
                if not math.isclose(expected_value, observed_value, rel_tol=1e-5, abs_tol=1e-5):
                    print(f"  ERROR: Key [{key_of_interest}] has unexpected value [{observed_value}]!")
                else:
                    print(f"  PASS: Key [{key_of_interest}] has expected value!")
            else:
                if expected_value is not observed_value:
                    print(f"  ERROR: Key [{key_of_interest}] has unexpected value [{observed_value}]!")
                else:
                    print(f"  PASS: Key [{key_of_interest}] has expected value!")

## WordCount - the "Hello World" of MapReduce

Before we ask you to write your own MapReduce program, you should first investigate the example from our lecture. 

Our input are two partitions of key value pairs denoting documents, where the document is the key, and the value is text of the document. 

In [ ]:
partitioned_documents = [
    # Input partition 0
    [
        (1, "the shawshank redemption"),
        (2, "the godfather")
    ],
    # Input partition 1
    [
        (3, "the godfather part two")
    ]
]

We want to count the number of occurrences of each word using mapreduce. The following code shows the corresponding functions `f_map` and `f_reduce`.  The `f_map` function split the text into words and emits a key value pair `(word, 1)` for each word it finds. The `f_reduce` function receives a word as key and its associated counts, and computes the total number of occurrences from that.

In [ ]:
def wordcount_f_map(doc_id, text):   
    output_tuples = []
    for word in text.split(" "):
        output_tuples.append((word, 1))
    return output_tuples        

def wordcount_f_reduce(word, counts):
    total_count = 0
    for count in counts:
        total_count += count
    return [(word, total_count)]

Next, we will execute the program in the next cell. The following diagram shows the flow of data during the execution of our MapReduce program. After execution, look at the log outputs of our engine and compare them to the diagram. Can you see the correspondence?

![title](wordcount.png)

In [ ]:
wordcount_result = mapreduce(partitioned_documents, wordcount_f_map, wordcount_f_reduce, print_debug_text=True)

Test your program by running the following cell, which checks whether your program produces the correct output for our example data.

In [ ]:
wordcount_result

In [ ]:
verify_locally(wordcount_result, 6, [("the", 3), ("godfather", 2), ("two", 1)])

## Task 1: Sensor Measurements

In the following, we ask you to write some mapreduce programs of your own. In the first task, we will work with sensor data. We have a couple of __sensors__ that have a name and produce some numerical value as measurement. Our data consists of several partitions, which contain key value pairs with the __name__ of the sensor as key and the __measurement__ as value. 

In [ ]:
partitioned_sensor_measurements = [
    [
        ("Sensor1", 12.3),
        ("Sensor2", 9.3),
        ("Sensor3", 2.7),
        ("BrokenSensor", 1000.0),
    ],
    [
        ("Sensor1", 17.1),
        ("Sensor2", 7.3),
        ("Sensor3", -12),
    ],
    [
        ("Sensor1", 19.1),
        ("BrokenSensor", 5000.0),
        ("Sensor2", 3.3),
        ("Sensor3", 0.0),
    ],    
]

### 1(a) - Compute the maximum measurement per sensor (except for the broken sensor)

Time to write your first mapreduce program. Your task is to compute the maximum measurement value per sensor, for all sensors except the "BrokenSensor". Implement your code in the `max_measurement_per_sensor_f_map` and `max_measurement_per_sensor_f_reduce` functions (and change the return statement).

Please note that the result of your mapreduce program should be equivalent to running the following SQL query on the data (if it was in a database):

```
SELECT sensor, MAX(measurement)
FROM sensors
WHERE name != "BrokenSensor"
GROUP BY sensor
```

In [ ]:
def a2_t1a_max_measurement_per_sensor():
    def max_measurement_per_sensor_f_map(sensor, measurement):   
        # IMPLEMENT ME
        if sensor.startswith("Broken"):
            return []
        return [(sensor, measurement)]

    
    def max_measurement_per_sensor_f_reduce(sensor, measurements):
        # IMPLEMENT ME
        max_measurement = max(measurements)
        return [(sensor, max_measurement)]
        
    
    return max_measurement_per_sensor_f_map, max_measurement_per_sensor_f_reduce

Execute your program by running the following cell: 

In [ ]:
result = mapreduce(partitioned_sensor_measurements, *a2_t1a_max_measurement_per_sensor(), print_debug_text=True)

Test your program by running the following cell, which checks whether your program produces the correct output for our example data.

In [ ]:
verify_locally(result, 3, [("Sensor1", 19.1), ("Sensor2", 9.3), ("Sensor3", 2.7)])

In [ ]:
am.test_student_function(a2_t1a_max_measurement_per_sensor)

### 1(b) - Compute the globally maximal measurement

In the next task, you have to compute the globally maximal measurement value from all sensors (including the "BrokenSensor"). Implement your code in the `global_max_measurement_f_map` and `global_max_measurement_f_reduce` functions (and change the return statement).

Hint: use an artificial key "global" to produce the final output.

In [ ]:
def a2_t1b_global_max_measurement():
    def global_max_measurement_f_map(sensor, measurement):   
        # IMPLEMENT ME
        return [("global", measurement)]
    
    def global_max_measurement_f_reduce(sensor, measurements):            
        # IMPLEMENT ME
        global_max = max(measurements)
        return [("global",global_max)]
    
    return global_max_measurement_f_map, global_max_measurement_f_reduce

Execute your program by running the following cell: 

In [ ]:
result = mapreduce(partitioned_sensor_measurements, *a2_t1b_global_max_measurement(), print_debug_text=True) 

Test your program by running the following cell, which checks whether your program produces the correct output for our example data.

In [ ]:
verify_locally(result, 1, [("global", 5000)])

In [ ]:
am.test_student_function(a2_t1b_global_max_measurement)

## Task 2 - Text Processing

In the next task, we implement some simple text processing program with mapreduce. Our input data is partitioned data about books, where each key value pair contains a __book_id__ as key and a __sentence__ as value.

In [ ]:
partitioned_books = [
    [
        (1, "It was a bright cold day in April, and the clocks were striking thirteen."),
        (2, "Mr. Jones, of the Manor Farm, had locked the hen-houses for the night, but was too drunk to remember to shut the pop-holes. "),
        (3, "He was a tough-looking youth of twenty-five or six, with reddish-yellow hair and powerful shoulders.")
    ],
    [
        (1, "On each landing, opposite the lift-shaft, the poster with the enormous face gazed from the wall."),
        (2, "All the animals were now present except Moses, the tame raven, who slept on a perch behind the back door."),        
        (3, "This was in late December 1936, less than seven months ago as I write, and yet it is a period that has already receded into enormous distance.")
    ]
]

### Compute the number of commas per book.

Please write a program to count the number of commas occurring in the text of each book. Implement your code in the `commacount_f_map` and `commacount_f_reduce` functions (and change the return statement).

Please note that the result of your mapreduce program should be equivalent to running the following SQL query on the data (if it was in a database and there was a CHAR_COUNT function):

```
SELECT book_id, SUM(CHAR_COUNT(sentence, ','))
FROM book_sentences
GROUP BY book_id
```


In [ ]:
def a2_t2_commacount():
    def commacount_f_map(book_id, sentence):   
        # IMPLEMENT ME
        commacount = 0
        for char in sentence:
            if char == ",":
                commacount += 1
        return [(book_id, commacount)]          


    def commacount_f_reduce(book_id, counts):
        # IMPLEMENT ME
        total_commacounts = sum(counts)
        return [(book_id,total_commacounts)]

    return commacount_f_map, commacount_f_reduce

Execute your program by running the following cell:

In [ ]:
result = mapreduce(partitioned_books, *a2_t2_commacount(), print_debug_text=True)

Test your program by running the following cell, which checks whether your program produces the correct output for our example data.

In [ ]:
verify_locally(result, 3, [(2, 5), (1, 3), (3, 3)])

In [ ]:
am.test_student_function(a2_t2_commacount)

### Task 3 - Movie visitor numbers

In the final mapreduce task of this assignment, we work with key value pairs about the visitor numbers of a movie for several weeks. Note that the data is a bit more messy than before. The key denotes the __name__ of the movie, but the value is a nested array of tuples, denoting the number of the __week__ and the __number of visitors__ in that week for a given movie.

In [ ]:
partitioned_movie_stats = [
    [
        ("MovieA", [(1, 100), (2, 20), (3, "50")]),
        ("MovieC", [(1, 100), (2, "250"), (3, 100), (4, "120")]),        
        ("MovieB", [(1, 1000), (2, 250)]),

    ],
    [
        ("MovieA", [(4, 50), (5, "10"), (6, 0)]),
        ("MovieB", [(3, 0), (4, "260")]),  
        ("MovieC", [(5, "180")]),
    ]
]

### Maximum number of visitors in a week per movie

Please write a mapreduce program to compute the maximum number of visitors in a week per movie (but ignore the first week of each movie). Implement your code in the `max_weekly_visitors_f_map` and `max_weekly_visitors_f_reduce` functions (and change the return statement).

In [ ]:
def a2_t3_max_weekly_visitors():
    def max_weekly_visitors_f_map(movie, weekly_stats):   
        # IMPLEMENT ME
        max_count = 0
        for week, count in weekly_stats:
            if week != 1:
                count = int(count)
                if count > max_count:
                    max_count = count
        return [(movie,max_count)]

    def max_weekly_visitors_f_reduce(movie, numbers):
        # IMPLEMENT ME
        max_weekly_visitors = max(numbers)
        return [(movie,max_weekly_visitors)]
    
    return max_weekly_visitors_f_map, max_weekly_visitors_f_reduce

Execute your program by running the following cell:

In [ ]:
result = mapreduce(partitioned_movie_stats, *a2_t3_max_weekly_visitors(), print_debug_text=True)

Test your program by running the following cell, which checks whether your program produces the correct output for our example data.

In [ ]:
verify_locally(result, 3, [('MovieA', 50), ('MovieB', 260), ('MovieC', 250)])

In [ ]:
am.test_student_function(a2_t3_max_weekly_visitors)

# Part B - Spark

In the second part of the assignment, we revisit our problems but solve them with a real dataflow system now: [Apache Spark](https://spark.apache.org/docs/latest/api/python/index.html). Please note that we use its Python variant in local mode for simplicity and easy debugging. But your programs would also run on a real cluster!


But first, we define other helper function to test our program outputs locally

In [ ]:
def verify_locally_from_rdd(result, expected_num_keys, expected_keys_and_values):
    verify_locally([result.collect()], expected_num_keys, expected_keys_and_values)

Spark has the concept of a session that allows us to move data into the cluster. We use it in local mode though.

Note that you can ignore the following warnings that might occur when executing the next cell:

```
WARNING: An illegal reflective access operation has occurred
WARNING: Illegal reflective access by org.apache.spark.unsafe.Platform (file:/usr/local/spark-3.2.0-bin-hadoop3.2/jars/spark-unsafe_2.12-3.2.0.jar) to constructor java.nio.DirectByteBuffer(long,int)
WARNING: Please consider reporting this to the maintainers of org.apache.spark.unsafe.Platform
WARNING: Use --illegal-access=warn to enable warnings of further illegal reflective access operations
WARNING: All illegal access operations will be denied in a future release
```

and

```
 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
```


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

spark = SparkSession.builder \
    .master("local") \
    .config("spark.driver.bindAddress","127.0.0.1") \
    .getOrCreate()

Spark works on so-called RDDS (resilient distributed datasets). We revisit our "hello world" example and create an RDD of documents.

In [ ]:
documents = spark.sparkContext.parallelize([
    (1, "the shawshank redemption"),
    (2, "the godfather"),
    (3, "the godfather part two")
], 2)

We write programs in Spark by RDDs into new RDDs. Similar to MapReduce we can write our own functions to apply to the data during these transformations. 

Be sure to consult the [overview over the PySpark RDD API](https://spark.apache.org/docs/latest/api/python/reference/pyspark.html#rdd-apis) for the following tasks.


The following code shows our "Hello World" example in Spark. We apply a `flatMap` to transform an document into multiple word tuples (analogously to our f_map function in mapreduce) and use Spark's `reduceByKey` transformation to group and reduce the resulting output. Note that we only have to define how to aggregate (e.g., sum) the values here.

In [ ]:
def extract_words(docid_and_title):
    text = docid_and_title[1]
    output_tuples = []
    for word in text.split(" "):
        output_tuples.append((word, 1))
    # return list of tuples [(token1,1),(token2,1),...]
    return output_tuples        

wordcounts = documents\
    .flatMap(lambda docid_and_title: extract_words(docid_and_title))\
    .reduceByKey(lambda count1, count2: count1 + count2)

We can use the adapted test function to verify that the outputs of our program are correct.

In [ ]:
verify_locally_from_rdd(wordcounts, 6, [("the", 3), ("godfather", 2), ("two", 1)])

Spark RDD's have a built-in action `collect` which transforms them into a Python collection. In real-world usecases, this would load the data from the cluster into the memory of our program (and potentially cause a crash), but we can use it here for debugging and inspecting the outputs of our transformations.

Here is how the data looks after the flatMap.

In [ ]:
documents\
    .flatMap(lambda docid_and_title: extract_words(docid_and_title))\
    .collect()

And this cell shows how the data looks like after the reduce:

In [ ]:
documents\
    .flatMap(lambda docid_and_title: extract_words(docid_and_title))\
    .reduceByKey(lambda count1, count2: count1 + count2)\
    .collect()

Note that we can often write Spark programs without having to explicitly define new functions, we can also use lambda functions and list comprehensions, like this:

In [ ]:
wordcounts = documents\
    .flatMap(lambda docid_and_title: [(word, 1) for word in docid_and_title[1].split(" ")])\
    .reduceByKey(lambda count1, count2: count1 + count2)

In [ ]:
verify_locally_from_rdd(wordcounts, 6, [("the", 3), ("godfather", 2), ("two", 1)])

### Task 4 - Sensors again

Next, we revisit our sensors task. First we load the measurement data into an RDD.


In [ ]:
sensor_measurements = spark.sparkContext.parallelize([
        ("Sensor1", 12.3),
        ("Sensor2", 9.3),
        ("Sensor3", 2.7),
        ("BrokenSensor", 1000.0),
        ("Sensor1", 17.1),
        ("Sensor2", 7.3),
        ("Sensor3", -12),
        ("Sensor1", 19.1),
        ("BrokenSensor", 5000.0),
        ("Sensor2", 3.3),
        ("Sensor3", 0.0)
], 2)

### Task 4(a) - Maximum value per sensor

Your task is to compute an RDD containing the maximum measurement value per sensor, for all sensors except the "BrokenSensor". Write the corresponding code in the `compute_max_value_per_sensor` function (and change the return statement).

Note that you can use the next cell for locally trying out some Spark code, before you fill in your solution into the `compute_max_value_per_sensor` function.


In [ ]:
# This is just for local development, you can test different transformations here
sensor_measurements\
    .filter(lambda sensor_measurement: sensor_measurement[0] != "BrokenSensor") \
    .reduceByKey(lambda a,b: max(a,b)) \
    .collect()


In [ ]:
def a2_t4a_compute_max_value_per_sensor(sensor_measurements):
    # IMPLEMENT ME
    return sensor_measurements\
    .filter(lambda sensor_measurement: sensor_measurement[0] != "BrokenSensor") \
    .reduceByKey(lambda a,b: max(a,b))

Execute the following cell to test your Spark program locally:

In [ ]:
result = a2_t4a_compute_max_value_per_sensor(sensor_measurements)

verify_locally_from_rdd(result, 3, [("Sensor1", 19.1), ("Sensor2", 9.3), ("Sensor3", 2.7)])

In [ ]:
am.test_student_function(a2_t4a_compute_max_value_per_sensor)

### Task 4(b) - Globally maximal sensor value

In the next task, you have to compute an RDD with the globally maximal measurement value from all sensors (including the "BrokenSensor"). Implement your code in the `compute_global_max_value` function (and change the return statement). Hint: use an artificial key "global" to produce the final output.

Note that you can use the next cell for locally trying out some Spark code, before you fill in your solution into the `compute_global_max_value` function.

In [ ]:
# This is just for local development, you can test different transformations here
sensor_measurements\
    .flatMap(lambda sensor_measurements: [("global",sensor_measurements[1])]) \
    .reduceByKey(lambda a,b: max(a,b)) \
    .collect()

In [ ]:
def a2_t4b_compute_global_max_value(sensor_measurements):
    # IMPLEMENT ME
    return sensor_measurements\
    .flatMap(lambda sensor_measurements: [("global",sensor_measurements[1])]) \
    .reduceByKey(lambda a,b: max(a,b))

Execute the following cell to test your Spark program locally:

In [ ]:
result = a2_t4b_compute_global_max_value(sensor_measurements)
verify_locally_from_rdd(result, 1, [("global", 5000)])

In [ ]:
am.test_student_function(a2_t4b_compute_global_max_value)

### Task 5 - Movie stats again

Next, we also revisit our movie stats task from before. First, we turn the data into an RDD:


In [ ]:
movie_stats = spark.sparkContext.parallelize([
    ("MovieA", [(1, 100), (2, 20), (3, "50")]),
    ("MovieC", [(1, 100), (2, "250"), (3, 100), (4, "120")]),        
    ("MovieB", [(1, 1000), (2, 250)]),
    ("MovieA", [(4, 50), (5, "10"), (6, 0)]),
    ("MovieB", [(3, 0), (4, "260")]),  
    ("MovieC", [(5, "180")]),
])

Please write a Spark program to compute an RDD containing the maximum amount of visitors per week per movie (but ignore the first week of each movie). Implement your code in the `compute_max_weekly_visitors` function (and change the return statement).

Note that you can use the next cell for locally trying out some Spark code, before you fill in your solution into the `compute_max_weekly_visitors` function.

In [ ]:
# This is just for local development, you can test different transformations here
movie_stats\
    .flatMap(lambda entry: [(entry[0], int(count)) for week, count in entry[1] if week != 1]) \
    .reduceByKey(lambda a,b: max(a,b)) \
    .collect()

In [ ]:
def a2_t5_compute_max_weekly_visitors(movie_stats):
    # IMPLEMENT ME    
    return movie_stats\
    .flatMap(lambda entry: [(entry[0], int(count)) for week, count in entry[1] if week != 1]) \
    .reduceByKey(lambda a,b: max(a,b))

Execute the following cell to test your Spark program locally:

In [ ]:
result = a2_t5_compute_max_weekly_visitors(movie_stats)
verify_locally_from_rdd(result, 3, [('MovieA', 50), ('MovieB', 260), ('MovieC', 250)])

In [ ]:
am.test_student_function(a2_t5_compute_max_weekly_visitors)

# Part C - Spark SQL

In the final part of this assignment, we explore how Spark unifies relational processing and dataflow computation via [SparkSQL](https://spark.apache.org/docs/3.2.1/sql-programming-guide.html). Spark has a distributed version of the well-known dataframes and allows us to query them with SQL.

Be sure to consult the [overview over PySpark's dataframe API](https://spark.apache.org/docs/3.2.1/api/python/reference/pyspark.sql.html#dataframe-apis) for the following tasks.


Let us first define two dataframes. They might remind you of the tables in the previous assignment.

In [ ]:
from pyspark.sql import Row

customers = spark.sparkContext.parallelize([
    Row(customer_id='1122334455', firstname='Ann', lastname='O\'Brien', city='Rotterdam'),
    Row(customer_id='1231231231', firstname='John', lastname='Doe', city='Amsterdam'), 
    Row(customer_id='1234567890', firstname='Paul', lastname='Murphy', city='Diemen'), 
    Row(customer_id='9876543210', firstname='Jack', lastname='Murphy', city='Utrecht'), 
    Row(customer_id='9999999999', firstname='Norah', lastname='Jones', city='Amsterdam')
], 2).toDF()  

sales = spark.sparkContext.parallelize([
    Row(customer_id='1122334455', model='2010', quantity=1, paid=2300, type_of_payment='mastercard credit'), 
    Row(customer_id='1122334455', model='3001', quantity=1, paid=99, type_of_payment='cash'), 
    Row(customer_id='1231231231', model='2002', quantity=2, paid=1898, type_of_payment='visa credit'), 
    Row(customer_id='1231231231', model='3002', quantity=1, paid=239, type_of_payment='cash'), 
    Row(customer_id='1234567890', model='1001', quantity=1, paid=1902, type_of_payment='mastercard credit'), 
    Row(customer_id='9876543210', model='1007', quantity=1, paid=510, type_of_payment='visa debit'), 
    Row(customer_id='9876543210', model='1007', quantity=3, paid=1530, type_of_payment='visa debit'), 
    Row(customer_id='9876543210', model='2002', quantity=1, paid=949, type_of_payment='visa debit'), 
    Row(customer_id='9999999999', model='1007', quantity=1, paid=459, type_of_payment='visa credit'), 
    Row(customer_id='9999999999', model='3007', quantity=2, paid=360, type_of_payment='visa credit')
], 2).toDF()     

Spark dataframes have a built-in method `show` which allows us to take a look at the first rows of the dataframe. We can use this to inspect our sales and customer data. It looks a lot like relational tables.

In [ ]:
customers.show()

In [ ]:
sales.show()

Now we can use the SparkSQL API to transform these dataframes with SQL. As an example, let's assume we are interested in the number of customers per city. In a relational database, we could query for this quantity like this:

```
SELECT city, COUNT(*)
FROM customers
GROUP BY city
```

Spark allows us to write a dataflow program that produces identical results to the query and also looks very much a like:

In [ ]:
cities_with_counts = customers\
  .groupby("city")\
  .count()

We can again use `show` to inspect the result.

In [ ]:
cities_with_counts.show()

Let's look at another example, what are the distinct types of payment in the sales data? In SQL, a corresponding query would look like this:

```
SELECT DISTINCT type_of_payment
FROM sales
```

Again, the corresponding Spark code is very similar:

In [ ]:
payment_types = sales\
    .select(['type_of_payment'])\
    .distinct()\
    .show()

Before we give you more tasks, we first define another helper function to verify SparkSQL results

In [ ]:
def verify_sql_result_locally(observed_dataframe, expected_rows):
    observed_rows = observed_dataframe.collect()
    if len(observed_rows) != len(expected_rows):
        print(f"ERROR: Expected {len(expected_rows)} rows in the result, but found {len(observed_rows)}!")
    
    for expected_row in expected_rows:
        if not expected_row in observed_rows:
            print(f"ERROR: Row {expected_row} missing from result!")
        else:    
            print(f"PASS: Encountered expected row {expected_row}")

### Task 6 - Filter customers

Write a SparkSQL program to find the distinct ids of all customers who made a visa credit payment of less than 1000 euros. Implement the `customers_with_certain_visa_payment` function, and make it return the correct Dataframe.

Note that you can use the next cell for locally trying out some Spark code, before you fill in your solution into the `customers_with_certain_visa_payment` function.

In [ ]:
# This is just for local development, you can test different transformations here
sales\
    .filter((sales.paid < 1000) & (sales.type_of_payment == "visa credit")) \
    .select(sales.customer_id)\
    .distinct()\
    .show()

In [ ]:
def a2_t6_customers_with_certain_visa_payment(sales):
    # IMPLEMENT ME    
    return sales\
    .filter((sales.paid < 1000) & (sales.type_of_payment == "visa credit")) \
    .select(sales.customer_id)\
    .distinct()

Execute the following cell to test your Spark SQL program locally:

In [ ]:
result = a2_t6_customers_with_certain_visa_payment(sales)
verify_sql_result_locally(result, [Row(customer_id='9999999999')])

In [ ]:
am.test_student_function(a2_t6_customers_with_certain_visa_payment)

### Task 7 - Payment types in Amsterdam

Write a SparkSQL program to find the payment types used by customers in a given city. No type should occur more than once. Implement the `find_payment_types` function to return the correct Dataframe. You can use Spark's `join` function to join the two tables, you can find the documentation [here](https://spark.apache.org/docs/3.2.1/api/python/reference/api/pyspark.sql.DataFrame.join.html?highlight=join#pyspark.sql.DataFrame.join).

Note that you can use the next cell for locally trying out some Spark code, before you fill in your solution into the `find_payment_types` function.

In [ ]:
# This is just for local development, you can test different transformations here
sales\
    .join(customers, on="customer_id")\
    .filter(customers.city=="Amsterdam")\
    .select(sales.type_of_payment)\
    .distinct()\
    .show()

In [ ]:
def a2_t7_find_payment_types(sales, city, customers):
    # IMPLEMENT ME
    return sales\
    .join(customers, on="customer_id")\
    .filter(customers.city=="Amsterdam")\
    .select(sales.type_of_payment)\
    .distinct()

Execute the following cell to test your Spark SQL program locally:

In [ ]:
result = a2_t7_find_payment_types(sales, "Amsterdam", customers)
verify_sql_result_locally(result, [Row(type_of_payment='visa credit'), Row(type_of_payment='cash')])

In [ ]:
am.test_student_function(a2_t7_find_payment_types)

### Task 8 - Number of sales and money per city

Write a spark SQL program that computes the amount of money paid per city.  Implement the `quantities_per_city` function to return the correct Dataframe with the columns __city__ and __amount__. To find the aggregation function you need for this, you can look at the `.groupby()` [documentation](https://spark.apache.org/docs/3.2.1/api/python/reference/api/pyspark.sql.DataFrame.groupBy.html). Depending on how you do this, you may want to use Spark's `withColumnRenamed` function to rename columns where appropriate.

Note that you can use the next cell for locally trying out some Spark code, before you fill in your solution into the `quantities_per_city` function.

In [ ]:
# This is just for local development, you can test different transformations here
import pyspark.sql.functions as F
sales\
    .join(customers, on="customer_id")\
    .groupBy("city")\
    .agg(F.sum("paid").alias('amount'))\
    .show()

In [ ]:
sales\
    .join(customers, on="customer_id")\
    .groupby("city")\
    .sum("paid").alias("amount")\
    .show()

In [ ]:
def a2_t8_quantities_per_city(sales, customers):
    # IMPLEMENT ME
    return sales\
    .join(customers, on="customer_id")\
    .groupBy("city")\
    .agg(sf.sum("paid").alias('amount'))

In [185]:
def a2_t8_quantities_per_city(sales, customers):
    # IMPLEMENT ME
    return sales\
    .join(customers, on="customer_id")\
    .groupBy("city")\
    .agg(sf.sum("paid").alias('amount'))

Execute the following cell to test your Spark SQL program locally:

In [186]:
result = a2_t8_quantities_per_city(sales, customers)

expected_rows = [
    Row(city='Diemen', amount=1902),
    Row(city='Rotterdam', amount=2399),
    Row(city='Amsterdam', amount=2956),
     Row(city='Utrecht', amount=2989)
]

verify_sql_result_locally(result, expected_rows)

PASS: Encountered expected row Row(city='Diemen', amount=1902)
PASS: Encountered expected row Row(city='Rotterdam', amount=2399)
PASS: Encountered expected row Row(city='Amsterdam', amount=2956)
PASS: Encountered expected row Row(city='Utrecht', amount=2989)


In [163]:
am.test_student_function(a2_t8_quantities_per_city)

Running local tests...


a2_t8_quantities_per_city successfully passed local tests
Running remote test...


Test was successful. Congratulations!
